In [35]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [59]:
# Load the dataset into a pandas DataFrame
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Separate the features (X) and target variable (y)
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']


X_test = test_data.copy()

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X.columns if
                    X[cname].nunique() < 10 and
                    X[cname].dtype == "object"]

# # Select numerical columns
numerical_cols = [cname for cname in X.columns if
                  X[cname].dtype in ['int64', 'float64']]

# categorical_cols = ['MSZoning','Street']
# Keep selected columns only
my_cols = categorical_cols + numerical_cols

X = X[my_cols]
X_test=X_test[my_cols]


In [63]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine the preprocessing steps using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the regression model
model = LinearRegression()

# Create the pipeline by combining the preprocessing and modeling steps
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

In [64]:
# Choose the appropriate cross-validation technique (e.g., k-fold cross-validation)
# Here, we'll use 5-fold cross-validation
cv = 5

# Use cross_val_score to perform cross-validation and obtain the evaluation scores
scores = cross_val_score(pipeline, X, y, scoring='neg_mean_squared_error', cv=cv)

# Convert the negative mean squared error scores to positive values
mse_scores = -scores

# Compute the mean and standard deviation of the cross-validation scores
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

# Print the mean and standard deviation of the cross-validation scores
print(f"Mean MSE: {mean_mse}")
print(f"Standard Deviation of MSE: {std_mse}")

/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

Mean MSE: 1467631775.690047
Standard Deviation of MSE: 699130513.5378724


/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

In [71]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# 
# # Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# # Evaluate the performance of the model on the validation set
val_predictions = pipeline.predict(X_val)
mse = mean_squared_error(y_val, val_predictions)
r2 = r2_score(y_val, val_predictions)
# 
# # Print the evaluation metrics
print(f"MSE on Validation Set: {mse}")
print(f"R-squared on Validation Set: {r2}")

MSE on Validation Set: 939788396.3264098
R-squared on Validation Set: 0.8774773887131653


/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

In [89]:
# use Random Forest, use cross validation, fold =5
from sklearn.ensemble import RandomForestRegressor

# Define the regression model
model2 = RandomForestRegressor(n_estimators=500, random_state=0)

# Create the pipeline by combining the preprocessing and modeling steps
pipeline_RFR = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model2)
])


# X_train, X_val, y_train, y_val
# pipeline_RFR.fit(X_train,y_train)


In [90]:
from sklearn.metrics import make_scorer, mean_absolute_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Define a custom scoring function using mean_absolute_error
mae_scorer = make_scorer(mean_absolute_error)

# Use cross_val_score to perform cross-validation and obtain the evaluation scores
scores_rfr = cross_val_score(pipeline_RFR, X, y, scoring=mae_scorer, cv=5)



# Print the mean and standard deviation of the cross-validation scores
print(f"Mean MAE: {np.mean(scores_rfr)}")
print(f"Standard Deviation of MAE: {np.std(scores_rfr)}")

Mean MAE: 17634.746338356166
Standard Deviation of MAE: 897.635505558542


In [91]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

# Use cross_validate to perform cross-validation and obtain the evaluation scores
cv_results = cross_validate(pipeline_RFR, X, y, scoring=mae_scorer, cv=StratifiedKFold(n_splits=5))


# Print the mean and standard deviation of the cross-validation scores
print(f"Mean MAE: {np.mean(cv_results['test_score'])}")
print(f"Standard Deviation of MAE: {np.std(cv_results['test_score'])}")


/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/model_selection/_split.py:665: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Mean MAE: 17396.10108767123
Standard Deviation of MAE: 834.7824502643747


In [78]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [80]:
model3= GradientBoostingRegressor(n_estimators=500, random_state=0)

# Preprocessing for numerical data
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])

my_pipeline_GBR = Pipeline(steps=[('preprocessor', preprocessor),
                               ('gbm', model3)
                               ])
my_pipeline_GBR.fit(X_train,y_train)
preds3 =my_pipeline_GBR.predict(X_val)

score = mean_absolute_error(y_val, preds3)

# Print the mean absolute error scores
print('MAE:', -score.mean())

/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

MAE: -16420.60278039026
